In [12]:
import torch_geometric as tg

from models.se3_transformer import Se3EquivariantTransformer
from utils.load_md17 import load_md17
import e3nn

import torch
import torch_geometric as tg
import pytorch_lightning as pl
import torchmetrics

from workflows.md17_experiment import MD17TransformerTask
import datetime as dt


In [5]:
### Prototype workflow

In [6]:
energy_model = Se3EquivariantTransformer.construct_from_number_of_channels_and_lmax(num_channels=16,
                                                                             l_max=2,
                                                                             num_features=9, # Derived from dataset
                                                                             num_attention_layers=4,
                                                                             num_attention_heads=4,
                                                                             radial_network_hidden_units=32
                                                                             )

C:\Users\niksm\Documents\CodeForUni\venvs\transformer_invariants\lib\site-packages\torch\jit\_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


In [7]:
batch_size=2
radius=2

data = load_md17(dataset_name='aspirin CCSD', dataset_dir='../real_datasets', radius=radius)

train_dataloader = tg.loader.DataLoader(data['train'], batch_size=batch_size, shuffle=True)
validation_dataloader = tg.loader.DataLoader(data['validation'], batch_size=batch_size)
test_dataloader = tg.loader.DataLoader(data['test'], batch_size=batch_size)

In [8]:
MD17TransformerTask.compute_energy_normalisers(dataset=data['train'])

(tensor([-837711.3125]), tensor(2015.1858))

In [22]:
timestamp = dt.datetime.now()
timestamp = dt.datetime.strftime(timestamp, '%Y-%m-%d_%H-%M')

In [23]:
logger = pl.loggers.CSVLogger(name="my_test", version=timestamp, save_dir='lightning_logs')
# Logs are saved to os.path.join(save_dir, name, version).

trainer = pl.Trainer(max_epochs=1, logger=logger)
task = MD17TransformerTask(energy_model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [24]:
# torch.autograd.set_detect_anomaly(True)

In [25]:
trainer.fit(task, train_dataloader, test_dataloader)

Loading `train_dataloader` to estimate number of stepping batches.
C:\Users\niksm\Documents\CodeForUni\venvs\transformer_invariants\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(

  | Name                | Type                      | Params
------------------------------------------------------------------
0 | energy_model        | Se3EquivariantTransformer | 672   
1 | energy_train_metric | MeanAbsoluteError         | 0     
2 | energy_valid_metric | MeanAbsoluteError         | 0     
3 | energy_test_metric  | MeanAbsoluteError         | 0     
4 | force_train_metric  | MeanAbsoluteError         | 0     
5 | force_valid_metric  | MeanAbsoluteError         | 0     
6 | fo

Sanity Checking: 0it [00:00, ?it/s]

C:\Users\niksm\Documents\CodeForUni\venvs\transformer_invariants\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

C:\Users\niksm\Documents\CodeForUni\venvs\transformer_invariants\lib\site-packages\pytorch_lightning\trainer\call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
